In [1]:
import pandas as pd
!pip install alpha_vantage autogluon
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.sectorperformance import SectorPerformances
import matplotlib.pyplot as plt
import requests
import os
from time import sleep
from autogluon.tabular import TabularDataset, TabularPredictor

API_KEY = os.getenv('API_KEY')
symbol = 'MSFT'

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
# # get sector overview for stock
# url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}'
# r = requests.get(url)
# sector = r.json()['Sector']
# data = r.json()
# print(sector)
# print(data)

In [3]:
# # ***************Sentiment would be a good source but can't seem to get more than 50 items from the API so won't go back
# # far enough???????
# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# # url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={API_KEY}
# url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&time_from=20040825T0130&apikey={API_KEY}'
# print(url)
# r = requests.get(url)

# print(data)

In [4]:
# Get json object with the intraday data and another with  the call's metadata
# For the default date string index behavior
# ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')
# ts_data, ts_meta_data = ts.get_intraday(symbol, interval='1min', outputsize='full')


# ts_data['4. close'].plot()
# plt.title(f'Intraday Times Series for the {symbol} stock (1 min)')
# plt.show()

In [5]:
# Adjusted daily data
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')
ts_data, ts_meta_data = ts.get_daily_adjusted(symbol, outputsize='full')


# ts_data['4. close'].plot()
# plt.title('Intraday Times Series for the GOOGL stock (1 min)')
# plt.show()

In [6]:
adjusted_data = ts_data
adjusted_data.tail()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
index,,,,,,,,,
5820,1999-11-05,91.81,92.87,90.50,91.56,28.692677,35083700.0,0.0,1.0
5821,1999-11-04,92.31,92.75,90.31,91.75,28.752219,27119700.0,0.0,1.0
5822,1999-11-03,92.94,93.50,91.50,92.00,28.830563,22258500.0,0.0,1.0
5823,1999-11-02,92.75,94.50,91.94,92.56,29.006053,23174500.0,0.0,1.0
5824,1999-11-01,93.25,94.19,92.12,92.37,28.946512,26630600.0,0.0,1.0


In [7]:
# compute the percent change for each day
adjusted_data['day_pcent_change'] = 100 * ((ts_data['4. close'] - ts_data['1. open']) / ts_data['1. open'])


# ******need to predict whether price for tomorrow will go up by 2% or greater - so need label - buy - 0 or 1
# today's close data will be used to predict whether to buy tomorrow
# so model cannot have the most recent row in it, since that would be today's data
# from which tomorrow's prediction will be made

# shift all the next days data by one which will be used to determine whether it should have been bought or not
adjusted_data['next_days_pcent_change'] = adjusted_data['day_pcent_change'].shift(+1)

In [8]:
# create the label - binary classification of whether to buy or not based on whether the stock went up 2% or more
# that day

def buy_or_not(row):
    if row.next_days_pcent_change >= 2:
        return 1
    else:
        return 0

adjusted_data['buy_tomorrow'] = adjusted_data.apply(buy_or_not, axis='columns')

# next_days_pcent_change will pollute the data set so drop it

adjusted_data.drop(columns='next_days_pcent_change',
                  inplace=True)

In [9]:
# technical indicators - some of these are premium

wma = [5, 10, 20, 40, 80]

# def concat_wma_df(df, period):
#     '''
#     concatenate the pandas dataframes being passed into the function
#     '''
    
wma_data = pd.DataFrame()

for period in wma:
    
    sleep(60) # avoid breaching the API request rate limit
    ti = TechIndicators(key=API_KEY, output_format='pandas')
    ti_data, ti_meta_data = ti.get_wma(symbol, interval='daily', time_period=period)
    wma_data[f'WMA_{period}'] = ti_data['WMA']

    
wma_data.describe() 

# *********get the resistance figures for the MAs?
# need to understand the relevance of the MAs better and what predictive power they are supposed to be happened so it can be represented properly




# ti_data.plot()
# plt.title('WMA indicator for  GOOGL stock (60 min)')
# plt.show()

,WMA_5,WMA_10,WMA_20,WMA_40,WMA_80
count,5821.000000,5816.000000,5806.000000,5786.000000,5746.000000
mean,61.785635,61.752447,61.684538,61.545283,61.262752
std,76.639326,76.594046,76.499953,76.319833,76.009414
min,11.584200,11.816000,12.305400,12.779900,13.332300
25%,19.110500,19.122050,19.121300,19.102625,19.044800
50%,23.248700,23.241250,23.162850,23.096100,22.850900
75%,59.927800,59.917775,59.883775,59.854425,59.752050
max,337.632900,335.865500,333.989200,330.869400,324.950200


In [10]:
# tech indicator data is received in the reverse order to price data so reverse it
rev_wma_data = wma_data[::-1]
rev_wma_data.reset_index(inplace=True)

# rev_ti_data.drop(index=rev_ti_data.index[0], 
#         axis=0, 
#         inplace=True)
rev_wma_data.head()

,date,WMA_5,WMA_10,WMA_20,WMA_40,WMA_80
0,2022-12-22,241.3107,244.7744,246.8134,244.9998,241.4351
1,2022-12-21,243.2727,246.5553,247.7550,245.1227,241.5350
2,2022-12-20,244.0073,247.2005,248.1053,244.9479,241.4871
3,2022-12-19,246.6267,248.3844,248.7072,244.9073,241.5120
4,2022-12-16,250.5000,249.9909,249.4413,244.9337,241.5818


In [11]:
for period in wma:
    adjusted_data[f'WMA_{period}'] = rev_wma_data[f'WMA_{period}']
#     these should be percentage diff to close as well?
    adjusted_data[f'WMA_{period}_pcent_diff_to_close'] = (adjusted_data['4. close'] - adjusted_data[f'WMA_{period}']) / adjusted_data['4. close']
    adjusted_data[f'WMA_{period}_pcent_diff_to_open'] = (adjusted_data['1. open'] - adjusted_data[f'WMA_{period}']) / adjusted_data['4. close']
    adjusted_data.drop(columns=f'WMA_{period}', inplace=True)

In [12]:
adjusted_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,WMA_5_pcent_diff_to_close,WMA_5_pcent_diff_to_open,WMA_10_pcent_diff_to_close,WMA_10_pcent_diff_to_open,WMA_20_pcent_diff_to_close,WMA_20_pcent_diff_to_open,WMA_40_pcent_diff_to_close,WMA_40_pcent_diff_to_open,WMA_80_pcent_diff_to_close,WMA_80_pcent_diff_to_open
index,,,,,,,,,,,,,,,,,,,,,
0,2022-12-22,241.255,241.990,233.87,238.19,238.19,28651664.0,0.0,1.0,-1.270440,...,-0.013102,-0.000234,-0.027643,-0.014776,-0.036204,-0.023336,-0.028590,-0.015722,-0.013624,-0.000756
1,2022-12-21,241.690,245.615,240.11,244.43,244.43,23690607.0,0.0,1.0,1.133684,...,0.004735,-0.006475,-0.008695,-0.019905,-0.013603,-0.024813,-0.002834,-0.014044,0.011844,0.000634
2,2022-12-20,239.400,242.910,238.42,241.80,241.80,25150802.0,0.0,1.0,1.002506,...,-0.009129,-0.019054,-0.022335,-0.032260,-0.026077,-0.036002,-0.013019,-0.022944,0.001294,-0.008632
3,2022-12-19,244.860,245.210,238.71,240.45,240.45,29696436.0,0.0,1.0,-1.801029,...,-0.025688,-0.007347,-0.032998,-0.014658,-0.034341,-0.016000,-0.018537,-0.000197,-0.004417,0.013924
4,2022-12-16,248.550,249.840,243.51,244.69,244.69,86101994.0,0.0,1.0,-1.553007,...,-0.023744,-0.007969,-0.021664,-0.005889,-0.019418,-0.003643,-0.000996,0.014779,0.012703,0.028478


In [13]:
# ****************
# Now get bollinger band details - need to study this more and figure out what it means
# https://www.tradingview.com/support/solutions/43000501972-bollinger-bands-width-bbw/

# calculate the bollinger band width then look at it the difference over the last 6 months
# Bollinger Bands Width = (Upper Band - Lower Band) / Middle Band

# look for: The low volatility period is followed by a surge in volatility and 
# price breaks through the Upper Band or falls through the Lower Band signifying a 
# change in the sideways movement and the beginning of a new directional trend.


# In a Bullish BBW Squeeze

# BBW drops. (In the example below, the threshold is 6% however this changes from security to security and 
#             timeframe to timeframe)
# Price breaks through the Upper Band which starts a new upward trend. Volatility also increases.


# In a Bearish BBW Squeeze

# BBW drops. (In the example below, the threshold is 9% however this changes from security to 
#             security and timeframe to timeframe).
# Price falls below the Lower Band which starts a new downward trend. Volatility also increases.
        
bb = TechIndicators(key=API_KEY, output_format='pandas')
bb_data, bb_meta_data = bb.get_bbands(symbol, interval='daily', time_period=20)

In [14]:
bb_data['bb_width'] = (bb_data['Real Upper Band'] - bb_data['Real Lower Band']) / bb_data['Real Middle Band']
bb_data.reset_index(inplace=True)
bb_data.head()

,date,Real Upper Band,Real Middle Band,Real Lower Band,bb_width
0,2022-12-22,259.2236,247.6080,235.9924,0.093822
1,2022-12-21,258.8618,248.0775,237.2932,0.086943
2,2022-12-20,258.8543,248.1075,237.3607,0.086630
3,2022-12-19,258.8380,248.1200,237.4020,0.086394
4,2022-12-16,258.7710,248.1585,237.5460,0.085530


In [15]:
adjusted_data['bb_width'] = bb_data['bb_width']
adjusted_data['bb_upper'] = bb_data['Real Upper Band']
adjusted_data['bb_lower'] = bb_data['Real Lower Band']
# add the below to percentages like day_pcent_change?
adjusted_data['pcent_diff_to_bb_upper'] = (adjusted_data['4. close'] - adjusted_data['bb_upper']) / adjusted_data['4. close']
adjusted_data['pcent_diff_to_bb_lower'] = (adjusted_data['4. close'] - adjusted_data['bb_lower']) / adjusted_data['4. close']

In [16]:
# # Don't seem to be able to get this with sufficient historical data
# sp = SectorPerformances(key=API_KEY, output_format='pandas')
# sp_data, sp_meta_data = sp.get_sector()
# sp_data.head()
# # sp_data['Rank A: Real-Time Performance'].plot(kind='bar')
# # plt.title('Real Time Performance (%) per Sector')
# # plt.tight_layout()
# # plt.grid()
# # plt.show()

In [17]:
macd = TechIndicators(key=API_KEY, output_format='pandas')
sleep(60)
macd_data, macd_meta_data = bb.get_macd(symbol, series_type='close', interval='daily')
macd_data.reset_index(inplace=True)
adjusted_data['macd_signal'] = macd_data['MACD_Signal']

In [18]:
stochrsi = TechIndicators(key=API_KEY, output_format='pandas')
sleep(60)
stochrsi_data, stochrsi_meta_data = bb.get_stochrsi(symbol, series_type='close', interval='daily', time_period=10, fastkperiod=6, fastdmatype=1)
stochrsi_data.reset_index(inplace=True)
adjusted_data['stochrsi_fast_k'] = stochrsi_data['FastK']
adjusted_data['stochrsi_fast_d'] = stochrsi_data['FastD']

In [19]:
ppo = TechIndicators(key=API_KEY, output_format='pandas')
sleep(60)
ppo_data, ppo_meta_data = bb.get_ppo(symbol, series_type='close', interval='daily', fastperiod=10, matype=1)
ppo_data.reset_index(inplace=True)
adjusted_data['ppo'] = ppo_data['PPO']

In [20]:
adjusted_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
index,,,,,,,,,,,,,,,,,,,,,
0,2022-12-22,241.255,241.990,233.87,238.19,238.19,28651664.0,0.0,1.0,-1.270440,...,-0.000756,0.093822,259.2236,235.9924,-0.088306,0.009226,1.9982,0.0000,9.0893,2.5600
1,2022-12-21,241.690,245.615,240.11,244.43,244.43,23690607.0,0.0,1.0,1.133684,...,0.000634,0.086943,258.8618,237.2932,-0.059043,0.029198,2.4094,26.0602,18.1785,2.2490
2,2022-12-20,239.400,242.910,238.42,241.80,241.80,25150802.0,0.0,1.0,1.002506,...,-0.008632,0.086630,258.8543,237.3607,-0.070531,0.018359,2.7368,8.8755,10.2969,2.1004
3,2022-12-19,244.860,245.210,238.71,240.45,240.45,29696436.0,0.0,1.0,-1.801029,...,0.013924,0.086394,258.8380,237.4020,-0.076473,0.012676,3.0710,0.0000,11.7183,2.0902
4,2022-12-16,248.550,249.840,243.51,244.69,244.69,86101994.0,0.0,1.0,-1.553007,...,0.028478,0.085530,258.7710,237.5460,-0.057546,0.029196,3.3309,0.0000,23.4365,2.3733


In [21]:
# clean up NaNs
final_data = adjusted_data.dropna().reset_index(drop=True)


In [22]:
final_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
0,2022-12-22,241.255,241.990,233.87,238.19,238.19,28651664.0,0.0,1.0,-1.270440,...,-0.000756,0.093822,259.2236,235.9924,-0.088306,0.009226,1.9982,0.0000,9.0893,2.5600
1,2022-12-21,241.690,245.615,240.11,244.43,244.43,23690607.0,0.0,1.0,1.133684,...,0.000634,0.086943,258.8618,237.2932,-0.059043,0.029198,2.4094,26.0602,18.1785,2.2490
2,2022-12-20,239.400,242.910,238.42,241.80,241.80,25150802.0,0.0,1.0,1.002506,...,-0.008632,0.086630,258.8543,237.3607,-0.070531,0.018359,2.7368,8.8755,10.2969,2.1004
3,2022-12-19,244.860,245.210,238.71,240.45,240.45,29696436.0,0.0,1.0,-1.801029,...,0.013924,0.086394,258.8380,237.4020,-0.076473,0.012676,3.0710,0.0000,11.7183,2.0902
4,2022-12-16,248.550,249.840,243.51,244.69,244.69,86101994.0,0.0,1.0,-1.553007,...,0.028478,0.085530,258.7710,237.5460,-0.057546,0.029196,3.3309,0.0000,23.4365,2.3733


In [23]:
# don't need the dates
final_data.drop(columns='index', inplace=True)


In [24]:
final_data = final_data.iloc[1: , :]
final_data.head(60)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,buy_tomorrow,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
1,241.690,245.6150,240.1100,244.43,244.430000,23690607.0,0.00,1.0,1.133684,0,...,0.000634,0.086943,258.8618,237.2932,-0.059043,0.029198,2.4094,26.0602,18.1785,2.2490
2,239.400,242.9100,238.4200,241.80,241.800000,25150802.0,0.00,1.0,1.002506,0,...,-0.008632,0.086630,258.8543,237.3607,-0.070531,0.018359,2.7368,8.8755,10.2969,2.1004
3,244.860,245.2100,238.7100,240.45,240.450000,29696436.0,0.00,1.0,-1.801029,0,...,0.013924,0.086394,258.8380,237.4020,-0.076473,0.012676,3.0710,0.0000,11.7183,2.0902
4,248.550,249.8400,243.5100,244.69,244.690000,86101994.0,0.00,1.0,-1.553007,0,...,0.028478,0.085530,258.7710,237.5460,-0.057546,0.029196,3.3309,0.0000,23.4365,2.3733
5,253.720,254.2000,247.3400,249.01,249.010000,35568093.0,0.00,1.0,-1.856377,0,...,0.048848,0.087794,258.8948,237.1212,-0.039696,0.047744,3.4275,4.5199,46.8730,2.7907
6,257.130,262.5880,254.3100,257.22,257.220000,35410933.0,0.00,1.0,0.035002,0,...,0.061028,0.090537,258.8545,236.4335,-0.006354,0.080812,3.3755,100.0000,89.2262,4.1675
7,261.690,263.9150,253.0700,256.92,256.920000,42196884.0,0.00,1.0,-1.822767,0,...,0.080096,0.086077,257.4716,236.2236,-0.002147,0.080556,3.2131,100.0000,78.4524,5.6964
8,247.445,252.5400,247.1700,252.51,252.510000,30665069.0,0.00,1.0,2.046920,0,...,0.026286,0.080117,255.9013,236.1890,-0.013430,0.064635,3.1094,100.0000,56.9048,6.8953
9,244.695,248.3100,244.1600,245.42,245.420000,20609743.0,0.00,1.0,0.296287,1,...,0.016591,0.076538,255.1448,236.3362,-0.039625,0.037013,3.1121,6.6896,13.8097,7.4044
10,244.840,248.7400,243.0600,247.40,247.400000,22611757.0,0.00,1.0,1.045581,0,...,0.017024,0.077556,255.1077,236.0612,-0.031155,0.045832,3.1835,25.2027,20.9298,7.9892


In [25]:
final_data.describe()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,buy_tomorrow,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
count,5745.000000,5745.000000,5745.000000,5745.000000,5745.000000,5.745000e+03,5745.000000,5745.000000,5745.000000,5745.000000,...,5745.000000,5745.000000,5745.000000,5745.000000,5745.000000,5745.000000,5745.000000,5745.000000,5745.000000,5745.000000
mean,72.807962,73.603230,71.996378,72.819205,62.195739,4.701931e+07,0.004273,1.000174,0.025743,0.069278,...,0.244050,0.106592,65.067715,58.607854,0.201030,0.277040,0.256347,51.197394,51.195877,0.275261
std,74.244320,75.020449,73.405018,74.253592,77.091624,2.766973e+07,0.055729,0.013193,1.522364,0.253948,...,0.198275,0.067616,80.923974,72.499986,0.209013,0.196998,1.533261,40.590551,30.680670,2.203070
min,15.200000,15.620000,14.870000,15.150000,11.406063,5.850800e+06,0.000000,1.000000,-7.485481,0.000000,...,-0.162658,0.014134,13.388200,10.901000,-0.383613,-0.011300,-8.061600,0.000000,0.005700,-11.559100
25%,27.300000,27.510000,27.100000,27.310000,19.047264,2.798542e+07,0.000000,1.000000,-0.715087,0.000000,...,0.099669,0.061794,19.993100,18.092300,0.048845,0.126255,-0.186800,3.645500,22.982500,-0.888700
50%,41.470000,41.910000,41.160000,41.600000,22.962847,4.134480e+07,0.000000,1.000000,0.000000,0.000000,...,0.201225,0.089235,24.214300,21.769700,0.164885,0.243644,0.128100,51.662400,51.344600,0.497500
75%,72.470000,73.130000,71.810000,72.560000,60.719689,5.853860e+07,0.000000,1.000000,0.788988,0.000000,...,0.310983,0.129917,61.192100,59.344300,0.270156,0.329049,0.479800,100.000000,79.805200,1.634900
max,344.620000,349.670000,342.200000,343.110000,339.886190,5.910522e+08,3.080000,2.000000,11.548556,1.000000,...,0.759023,0.565266,347.515000,325.587700,0.699829,0.800273,9.067000,100.000000,99.957900,8.862000


In [26]:
train_data = final_data
label = 'buy_tomorrow'
save_path = './autogluon_models'

In [27]:
# there is currently a bug where num_bag_sets >1 produces an error with various algos - waiting on fix
# https://github.com/autogluon/autogluon/issues/2581

predictor = TabularPredictor(label=label, path=save_path, verbosity=3, eval_metric='precision').fit(
    train_data, presets='best_quality', time_limit=15000, num_bag_folds=20
)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Saving ./autogluon_models/learner.pkl
Saving ./autogluon_models/predictor.pkl
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to

In [29]:
results = predictor.fit_summary()

Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBM_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/CatBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./autogluon_models/models/XGBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl


*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           LightGBM_BAG_L1   0.932115       0.056746   4.672113                0.056746           4.672113            1       True          4
1         LightGBMXT_BAG_L1   0.932115       0.068025   3.083360                0.068025           3.083360            1       True          3
2            XGBoost_BAG_L1   0.931941       0.133424   5.245907                0.133424           5.245907            1       True         11
3    NeuralNetFastAI_BAG_L1   0.931767       0.245266  12.063515                0.245266          12.063515            1       True         10
4           CatBoost_BAG_L1   0.931593       0.012038   7.926146                0.012038           7.926146            1       True          7
5      LightGBMLarge_BAG_L1   0.931419       0.032439   8.601035                

In [30]:
predictor.leaderboard(train_data, silent=True)

Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBM_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/CatBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./autogluon_models/models/XGBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr_BAG_L1,1.000000,0.928285,0.219546,0.273283,2.300036,0.219546,0.273283,2.300036,1,True,6
1,RandomForestGini_BAG_L1,1.000000,0.929156,0.224756,0.279972,1.714419,0.224756,0.279972,1.714419,1,True,5
2,ExtraTreesEntr_BAG_L1,1.000000,0.928982,0.228308,0.276880,0.697282,0.228308,0.276880,0.697282,1,True,9
3,ExtraTreesGini_BAG_L1,1.000000,0.929678,0.240077,0.279397,0.698774,0.240077,0.279397,0.698774,1,True,8
4,KNeighborsDist_BAG_L1,1.000000,0.899391,0.958601,0.876894,0.010072,0.958601,0.876894,0.010072,1,True,2
5,WeightedEnsemble_L2,0.953699,0.930722,1.021211,1.143958,1.768880,0.003302,0.008215,1.059536,2,True,14
6,XGBoost_BAG_L1,0.938903,0.931941,0.121912,0.133424,5.245907,0.121912,0.133424,5.245907,1,True,11
7,CatBoost_BAG_L1,0.932811,0.931593,0.055521,0.012038,7.926146,0.055521,0.012038,7.926146,1,True,7
8,KNeighborsUnif_BAG_L1,0.931593,0.927589,0.777832,0.856346,0.010570,0.777832,0.856346,0.010570,1,True,1
9,NeuralNetFastAI_BAG_L1,0.930896,0.931767,1.789387,0.245266,12.063515,1.789387,0.245266,12.063515,1,True,10


In [31]:
predictor.feature_importance(train_data)

Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
Computing feature importance via permutation shuffling for 28 features using 5000 rows with 5 shuffle sets...
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
	146.68s	= Expected runtime (29.34s per shuffle set)
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/Weight

,importance,stddev,p_value,n,p99_high,p99_low
6. volume,0.02076,0.000607,8.739231e-08,5,0.022009,0.019511
WMA_5_pcent_diff_to_open,0.01376,0.001276,8.771092e-06,5,0.016387,0.011133
bb_width,0.01336,0.001178,7.181989e-06,5,0.015786,0.010934
WMA_20_pcent_diff_to_close,0.01268,0.000110,6.683801e-10,5,0.012906,0.012454
WMA_10_pcent_diff_to_open,0.01268,0.000965,4.001169e-06,5,0.014668,0.010692
WMA_5_pcent_diff_to_close,0.01236,0.000740,1.536718e-06,5,0.013884,0.010836
WMA_10_pcent_diff_to_close,0.01188,0.000460,2.702203e-07,5,0.012828,0.010932
pcent_diff_to_bb_lower,0.01188,0.000729,1.696521e-06,5,0.013382,0.010378
bb_lower,0.01172,0.000610,8.769808e-07,5,0.012976,0.010464
pcent_diff_to_bb_upper,0.01168,0.000593,7.961641e-07,5,0.012902,0.010458
